- ## *import packages*

In [2]:
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import emoji

- ## *check the package version*

In [4]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.26.4
Pandas:  2.1.4


 # Manipulating the data to make it usable

***

##### All the preprocessing of data for Mini Project 1 is documented here

***

- ### Wrangling
    - - #### sourcing, loading, and precleaning the data so we can see what it really looks like
    - -  #### fixing critical issues
    - - #### understanding the essential characteristics of the data
    - - #### applying preliminary transformations to confer context and meaning
    - - #### implementing strategies for missing and invalid data

- ## munging
    - - #### reshaping the data to prepare it for analysis 

- ## Profiling and cleaning
***

- #### *In tidy data:*
- ##### *1. Each variable forms a column.*
- ##### *2. Each observation forms a row.*
- ##### *3. Each type of observational unit forms a table.*

#### *1. Load the "watson healthcare" file into a data frame*
***

- #### *Connect to SQLite database*

In [15]:
conn = sqlite3.connect('healthcare.db')

- ## *Create a Data Frame using SQL*

In [17]:
# assign data frame name to "df" and store it with observed data
df = pd.read_csv(r'C:\Users\3sthe\Documents\MINI_PROJECT_1\employee-attrition-for-healthcare\watson_healthcare_modified.csv')

# Write DataFrame to SQLite using connection 'conn'
df.to_sql('healthcare', conn, if_exists='replace', index = False)

# Query data from SQLite table
query = """
SELECT *
FROM healthcare;
"""
# assign new data frame name to "df_from_sql" and use the query to load df table into df_from_sql
df_from_sql = pd.read_sql_query(query, conn)

- ## *Check that file was loaded correctly*

In [19]:
# observe the first column
df_from_sql.iloc[0]

EmployeeID                        1313919
Age                                    41
Attrition                              No
BusinessTravel              Travel_Rarely
DailyRate                            1102
Department                     Cardiology
DistanceFromHome                        1
Education                               2
EducationField              Life Sciences
EmployeeCount                           1
EnvironmentSatisfaction                 2
Gender                             Female
HourlyRate                             94
JobInvolvement                          3
JobLevel                                2
JobRole                             Nurse
JobSatisfaction                         4
MaritalStatus                      Single
MonthlyIncome                        5993
MonthlyRate                         19479
NumCompaniesWorked                      8
Over18                                  Y
OverTime                              Yes
PercentSalaryHike                 

***
## *Insights*
- #### *Discussion about which columns are meaningless*
    - ##### *REASONS FOR DROPPING COLUMNS:*
    - ##### *EmployeeID*
        - ###### percieved as meaningless
    - ##### *Age; Gender; DistanceFromHome; MaritalStatus; OverTime; Shift*
        - ###### collectively insignificant 
    - ##### *BusinessTravel; DailyRate; HourlyRate; MonthlyRate; PercentSalaryHike*
        - ###### interchangeable and generalised by MonthlyIncome
    - ##### *EnvironmentSatisfaction; JobInvolvement; JobLevel; JobSatisfaction; RelationshipSatisfaction; WorkLifeBalance*
        - ###### interchangeable and generalised by PerformanceRating
    - ##### *YearsSinceLastPromotion; YearsWithCurrManager*
        - ###### interchangeable and generalised by YearsAtCompany
    - ##### *EmployeeCount; Over18; StandardHours*
        - ###### single value for every observation
***

#### *2. drop meaningless columns and create a filtered data frame*
***

In [22]:
# dropping multiple columns from a DataFrame
columns_to_drop = ['EmployeeID', 'Age', 'Gender', 'DistanceFromHome', 'MaritalStatus', 
                   'OverTime', 'Shift', 'BusinessTravel', 'DailyRate', 'HourlyRate', 
                   'MonthlyRate', 'PercentSalaryHike', 'EnvironmentSatisfaction', 
                   'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
                   'RelationshipSatisfaction', 'WorkLifeBalance', 
                   'YearsSinceLastPromotion', 'YearsWithCurrManager', 
                   'EmployeeCount', 'Over18', 'StandardHours']

df_main = df_from_sql.drop(columns=columns_to_drop)
# print the result
print(df_main)

     Attrition  Department  Education    EducationField    JobRole  \
0           No  Cardiology          2     Life Sciences      Nurse   
1           No   Maternity          1     Life Sciences      Other   
2          Yes   Maternity          2             Other      Nurse   
3           No   Maternity          4     Life Sciences      Other   
4           No   Maternity          1           Medical      Nurse   
...        ...         ...        ...               ...        ...   
1671       Yes   Neurology          3  Technical Degree      Nurse   
1672        No  Cardiology          3         Marketing      Nurse   
1673        No   Maternity          3     Life Sciences      Other   
1674        No   Neurology          1     Life Sciences  Therapist   
1675        No  Cardiology          2           Medical      Nurse   

      MonthlyIncome  NumCompaniesWorked  PerformanceRating  TotalWorkingYears  \
0              5993                   8                  3                  8 

- ## *Check that columns are loaded correctly*

In [24]:
## print the first 5 rows of obervations 
print(df_main.head(5))

  Attrition  Department  Education EducationField JobRole  MonthlyIncome  \
0        No  Cardiology          2  Life Sciences   Nurse           5993   
1        No   Maternity          1  Life Sciences   Other           5130   
2       Yes   Maternity          2          Other   Nurse           2090   
3        No   Maternity          4  Life Sciences   Other           2909   
4        No   Maternity          1        Medical   Nurse           3468   

   NumCompaniesWorked  PerformanceRating  TotalWorkingYears  \
0                   8                  3                  8   
1                   1                  4                 10   
2                   6                  3                  7   
3                   1                  3                  8   
4                   9                  3                  6   

   TrainingTimesLastYear  YearsAtCompany  YearsInCurrentRole  
0                      0               6                   4  
1                      3              

***
#### *3. observe each columns' unique values*
***
#### Summarising Qualitative Data
## *Insights*
- #### *Discussion about which columns to be observed*
    - ##### *REASONS FOR SKIPPING COLUMNS:*
    - ##### *Attrition*
        - ###### we know it has plural values 'Yes' and 'No'
    - ##### *Education; MonthlyIncome; PerformanceRating; TrainingTimesLastYear*
        - ###### continuous variables which would be useful directly for EDA
    - ##### *NumCompaniesWorked; TotalWorkingYears; YearsAtCompany; YearsInCurrentRole*
        - ###### continuous variables which will be useful in a time series
***

In [26]:
# Printing the value count of each Department 
print(df_from_sql['Department'].value_counts())

Department
Maternity     796
Cardiology    531
Neurology     349
Name: count, dtype: int64


In [27]:
# Printing the value count of each EducationField 
print(df_from_sql['EducationField'].value_counts())

EducationField
Life Sciences       697
Medical             524
Marketing           189
Technical Degree    149
Other                88
Human Resources      29
Name: count, dtype: int64


In [28]:
# Printing the value count of each JobRole
print(df_from_sql['JobRole'].value_counts())

JobRole
Nurse             822
Other             534
Therapist         189
Administrative    115
Admin              16
Name: count, dtype: int64


In [29]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


##### *'Admin' and 'Administrative' are alike*

In [31]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


In [32]:
# finding the Administrative values in the original df with a condition 'con'
con = (df_from_sql['JobRole']== 'Administrative')
# observing the total number of Administrative values to be added to the Admin column
print("Number of Administrative employees:", con.sum())

# finding the Admin values in the original df with a condition 'con'
con = (df_from_sql['JobRole']== 'Admin')
# observing the total number of Admin values
print("Number of Admin employees:", con.sum())

Number of Administrative employees: 115
Number of Admin employees: 16


In [33]:
# Define a mapping dictionary for merging categories
merge_map = {
    'Administrative': 'Admin'
}

# Use replace method with the mapping dictionary
df_main['JobRole'] = df_main['JobRole'].replace(merge_map)
print(df_main['JobRole'].value_counts())

JobRole
Nurse        822
Other        534
Therapist    189
Admin        131
Name: count, dtype: int64


***
##### *Qualitative variables are now ready for EDA*
***

***
#### *4. observe each columns' description*
***
#### Summarising Quantitative Data
## *Insights*
- #### *Discussion about which columns to be observed*
    - ##### *Education*
        - #### Education refers to Level of Education where '1' = Level 1 Diploma, '5' = Foundation Degree/Higher Level Diploma
    - ##### *TrainingTimesLastYear*
        - #### TrainingTimesLastYear refers to Training Sessions Last Year i.e '1' training session is all training completed within a particular day 
    - ##### *MonthlyIncome; PerformanceRating;*
        - ###### continuous variables which would be useful directly for EDA
    - ##### *NumCompaniesWorked; TotalWorkingYears; YearsAtCompany; YearsInCurrentRole*
        - ###### continuous variables which will be useful in a time series
***

In [36]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


- #### *Change Education and TrainingTimesLastYear column names to more meaningul names*
    - ##### Education == 'LevelOfEducation'
    - ##### TrainingTimesLastYear == 'TrainingSessionsLastYear'

In [38]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


In [39]:
# Renaming columns
df_main.rename(columns={'Education': 'LevelOfEducation', 'TrainingTimesLastYear': 'DaysofTrainingLastYear'}, inplace=True)

# Observe the data frame
print("\nDataFrame after renaming columns:")
print(df_main)


DataFrame after renaming columns:
     Attrition  Department  LevelOfEducation    EducationField    JobRole  \
0           No  Cardiology                 2     Life Sciences      Nurse   
1           No   Maternity                 1     Life Sciences      Other   
2          Yes   Maternity                 2             Other      Nurse   
3           No   Maternity                 4     Life Sciences      Other   
4           No   Maternity                 1           Medical      Nurse   
...        ...         ...               ...               ...        ...   
1671       Yes   Neurology                 3  Technical Degree      Nurse   
1672        No  Cardiology                 3         Marketing      Nurse   
1673        No   Maternity                 3     Life Sciences      Other   
1674        No   Neurology                 1     Life Sciences  Therapist   
1675        No  Cardiology                 2           Medical      Nurse   

      MonthlyIncome  NumCompaniesWorked 

***
##### *Quantitative variables are now ready for EDA*
***

***
#### *5. Checking for duplicate values*
***

In [42]:
# Check for any duplicates in the entire DataFrame using duplicated() method which iterates over the rows
if df_main.duplicated().any():
    print("There are duplicate values in the DataFrame.")
else:
    print("There are no duplicate values in the DataFrame.")

There are duplicate values in the DataFrame.


In [43]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


#### Find out if we can ignore the duplicate values claim

In [45]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


In [46]:
# Finding duplicate rows
duplicate_rows = df_main[df_main.duplicated()]

# check out the rows
print("\nDuplicate Rows:")
print(duplicate_rows)


Duplicate Rows:
     Attrition  Department  LevelOfEducation    EducationField    JobRole  \
1467        No  Cardiology                 4     Life Sciences      Admin   
1475        No  Cardiology                 4  Technical Degree      Nurse   
1477        No   Maternity                 1           Medical  Therapist   
1478        No   Neurology                 3   Human Resources      Admin   
1481        No  Cardiology                 2         Marketing      Nurse   
...        ...         ...               ...               ...        ...   
1658       Yes  Cardiology                 3           Medical      Other   
1659       Yes  Cardiology                 2           Medical      Nurse   
1660       Yes  Cardiology                 3     Life Sciences      Other   
1662       Yes   Neurology                 4     Life Sciences      Other   
1664       Yes  Cardiology                 4           Medical      Nurse   

      MonthlyIncome  NumCompaniesWorked  PerformanceRating

#### Duplicate values claim ignored as the check revealed duplicate values within columns and no row duplication...

***
#### *6. Checking for missing values*
*** 

In [49]:
# using function isnull() to find any null values in the observed data
df_main.columns[df_main.isnull().any()]

Index([], dtype='object')

In [50]:
# using function isna() to find any NaN values in the observed data
df_main.isna()

,Attrition,Department,LevelOfEducation,EducationField,JobRole,MonthlyIncome,NumCompaniesWorked,PerformanceRating,TotalWorkingYears,DaysofTrainingLastYear,YearsAtCompany,YearsInCurrentRole
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1671,False,False,False,False,False,False,False,False,False,False,False,False
1672,False,False,False,False,False,False,False,False,False,False,False,False
1673,False,False,False,False,False,False,False,False,False,False,False,False
1674,False,False,False,False,False,False,False,False,False,False,False,False


***
#### *7. Describing the data*
***

In [52]:
# print the statistics of the dataset
df_main.describe()

,LevelOfEducation,MonthlyIncome,NumCompaniesWorked,PerformanceRating,TotalWorkingYears,DaysofTrainingLastYear,YearsAtCompany,YearsInCurrentRole
count,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000
mean,2.907518,6516.512530,2.662291,3.150358,11.338902,2.805489,7.033413,4.264916
std,1.025835,4728.456618,2.477704,0.357529,7.834996,1.288431,6.098991,3.627456
min,1.000000,1009.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2928.250000,1.000000,3.000000,6.000000,2.000000,3.000000,2.000000
50%,3.000000,4899.000000,2.000000,3.000000,10.000000,3.000000,5.000000,3.000000
75%,4.000000,8380.250000,4.000000,3.000000,15.000000,3.000000,10.000000,7.000000
max,5.000000,19999.000000,9.000000,4.000000,40.000000,6.000000,40.000000,18.000000


##### This descriptive statistics table above gives insight into the distribution of the data which guides us into the next phase in data preprocessing: EDA...

In [54]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


- #### *'NumCompaniesWorked' min value = 0 contradicts an employee existing at this company*
    - ##### Manipulate 'NumCompaniesWorked'== 0
    - ##### result: 'NumCompaniesWorked'== 1

In [56]:
print(emoji.emojize("\U00002757"), emoji.emojize("\U00002757"), emoji.emojize("\U00002757"))

❗ ❗ ❗


In [57]:
# Change all 0 values in 'NumCompaniesWorked' to 1
df_main['NumCompaniesWorked'] = df_main['NumCompaniesWorked'].replace(0, 1)

# check that the 0's have been replaced with 1's
print(df_main['NumCompaniesWorked'].value_counts())

NumCompaniesWorked
1    824
3    180
2    173
4    156
7     84
6     79
5     70
9     59
8     51
Name: count, dtype: int64


# DATA MANIPULATION COMPLETE...

***
#### *8. Understanding the data*
***

In [60]:
# checking the index
df_main.index

RangeIndex(start=0, stop=1676, step=1)

In [61]:
# print the column names
print(df_main.columns.tolist())

# print the shape (number of rows, number of columns) of the df_main
df_main.shape

['Attrition', 'Department', 'LevelOfEducation', 'EducationField', 'JobRole', 'MonthlyIncome', 'NumCompaniesWorked', 'PerformanceRating', 'TotalWorkingYears', 'DaysofTrainingLastYear', 'YearsAtCompany', 'YearsInCurrentRole']


(1676, 12)

# DATA PROFILING COMPLETE...